In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.0 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [ ]:
!pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.4 MB/s eta 0:00:00


In [ ]:
import openai
import re
import time

import numpy as np
import random

from tqdm import tqdm
from datasets import load_dataset


In [ ]:
dataset = load_dataset("hotpot_qa", "fullwiki", trust_remote_code=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


The repository for hotpot_qa contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/hotpot_qa.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/90447 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/7405 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7405 [00:00<?, ? examples/s]

In [ ]:
train_data = dataset['train']
train_data

["Arthur's Magazine",
 'Delhi',
 'President Richard Nixon',
 'American',
 'alcohol',
 'Jonathan Stark',
 'Crambidae',
 'Badr Hari',
 '2006',
 '6.213 km long',
 'Jaime Meline',
 'Walter Darwin Coy',
 'United States',
 'Super Bowl XLVIII',
 'US 60',
 '2006',
 'Hetfield and Ulrich, longtime lead guitarist Kirk Hammett, and bassist Robert Trujillo.',
 'Fox',
 '2017',
 'Nevada',
 'Hawaii',
 'Kelli Ward',
 'The Wolfhounds',
 '16-year-old',
 'World War II',
 'Todd Phillips',
 'Carol Lawrence',
 'no',
 'no',
 'New York City',
 'Amy Jo Johnson',
 'Aleksander Ford',
 'yes',
 'director',
 'Roseau, Minnesota, USA',
 'The Saimaa Gesture',
 'David Lee Roth',
 'Nassau County',
 'Australia',
 'California',
 'Dessau',
 'Roman Catholic',
 'The Joshua Tree',
 'Ulster County',
 'Tammy Wynette',
 'Sir Francis Nethersole',
 'ingredients in beer',
 'Dennis Howard Marks',
 'Robert Sheehan',
 'no',
 'Glenn Hughes',
 'March 28, 1941',
 'standard gauge track',
 'Sex and the City',
 'Kato',
 'England',
 'Province

In [ ]:
prompt = 'Question: Currently, which division does the fighter, who lost to Jon Jones in the UFC 140 event, fight in?\
          Answer: Let\'s think step by step. UFC 140 was was a mixed martial arts \
          event on December 10, 2011 at the Air Canada Centre in Toronto, Ontario, \
          Canada. UFC 140 contained a fight between Jon Jones and Lyoto Machida. \
          In the main event, Lyoto Machida also suffered his first ever submission \
          loss at the hands of Jon Jones. \
          Lyoto Carvalho Machida born May 30, 1978) is a \
          Brazilian mixed martial artist who currently competes in the middleweight\
           division of the Ultimate Fighting Championship.\
          Therefore, Machida competes in the Middleweight division'

In [ ]:
index = int(random.random() * len(train_data))
question = train_data['question'][index]

prompt_q = prompt + '\n' + question
prompt_q

"Question: Currently, which division does the fighter, who lost to Jon Jones in the UFC 140 event, fight in?          Answer: Let's think step by step. UFC 140 was was a mixed martial arts           event on December 10, 2011 at the Air Canada Centre in Toronto, Ontario,           Canada. UFC 140 contained a fight between Jon Jones and Lyoto Machida.           In the main event, Lyoto Machida also suffered his first ever submission           loss at the hands of Jon Jones.           Lyoto Carvalho Machida (] ; 町田龍太) (born May 30, 1978) is a           Brazilian mixed martial artist who currently competes in the middleweight           division of the Ultimate Fighting Championship.          Therefore, Machida competes in the Middleweight division\nWhat year was the director of Tin Toy born?"

In [ ]:
import os
os.environ["OPENAI_API_KEY"] =

In [ ]:
client = openai.OpenAI()


completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "Follow the given examples and answer the question."},
        {
            "role": "user",
            "content": prompt_q
        }
    ]
)

print(completion.choices[0].message.content)


Let's think step by step. "Tin Toy" is a short animated film produced by Pixar Animation Studios and directed by John Lasseter. John Lasseter was born on January 12, 1957. Therefore, the director of "Tin Toy," John Lasseter, was born in 1957.


In [ ]:
print(completion.choices[0].message.content[-40:])
answer = train_data['answer'][index]
print(answer)

n Toy," John Lasseter, was born in 1957.
1957


In [ ]:
train_data['answer']

["Arthur's Magazine",
 'Delhi',
 'President Richard Nixon',
 'American',
 'alcohol',
 'Jonathan Stark',
 'Crambidae',
 'Badr Hari',
 '2006',
 '6.213 km long',
 'Jaime Meline',
 'Walter Darwin Coy',
 'United States',
 'Super Bowl XLVIII',
 'US 60',
 '2006',
 'Hetfield and Ulrich, longtime lead guitarist Kirk Hammett, and bassist Robert Trujillo.',
 'Fox',
 '2017',
 'Nevada',
 'Hawaii',
 'Kelli Ward',
 'The Wolfhounds',
 '16-year-old',
 'World War II',
 'Todd Phillips',
 'Carol Lawrence',
 'no',
 'no',
 'New York City',
 'Amy Jo Johnson',
 'Aleksander Ford',
 'yes',
 'director',
 'Roseau, Minnesota, USA',
 'The Saimaa Gesture',
 'David Lee Roth',
 'Nassau County',
 'Australia',
 'California',
 'Dessau',
 'Roman Catholic',
 'The Joshua Tree',
 'Ulster County',
 'Tammy Wynette',
 'Sir Francis Nethersole',
 'ingredients in beer',
 'Dennis Howard Marks',
 'Robert Sheehan',
 'no',
 'Glenn Hughes',
 'March 28, 1941',
 'standard gauge track',
 'Sex and the City',
 'Kato',
 'England',
 'Province